In [30]:
import pandas as pd
import numpy as np
import bamboolib as bam
from json import loads,dumps
import plotly.express as px

In [2]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\User\AppData\Local\Temp\ipykernel_1360\3509738684.py:3: DeprecationWarning:

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display



In [3]:
df = pd.read_excel(r'C:\Users\User\Desktop\myGPT\data\17052023_ver1.xlsx')
df

       PO Identifier  Vendor Number                           Vendor Name  \
0                768      1142981.0                              ExLibris   
1                769        58066.0                              ExLibris   
2              10467      1002161.0       Alliant Insurance Services, Inc   
3              10474        18030.0       Alliant Insurance Services, Inc   
4              12459      1065902.0                                   EMC   
...              ...            ...                                   ...   
30420          30421      1000633.0          Technology Integration Group   
30421          30422        56613.0                      Capital Datacorp   
30422          30423        17520.0                    THE PRIMARY SOURCE   
30423          30424        56613.0                                  AT&T   
30424          30425         2472.0  Enterprise Networking Solutions, Inc   

           PO Number  PO Rev  Line  PO Status  Buyer Name  \
0           P1

In [4]:
df = df.drop(columns=['PO Rev', 'Line', 'PO Status', 'Buyer Name', 'Delivery Date (Per PO)', 'Currency', 'PR Number','Group','Flagged','Family Title','PO Identifier','Vendor Number','Local Currency','Total PO Value Local Currency'])

In [5]:
df['PO Order Date'] = pd.to_datetime(df['PO Order Date'], format='%Y-%m-%d %H:%M:%S',errors='coerce')
df['PO Approval Date'] = pd.to_datetime(df['PO Approval Date'], format='%Y-%m-%d %H:%M:%S',errors='coerce')

In [6]:
df = df.loc[(df['PO Order Date'].notna()) & (df['PO Approval Date'].notna())]
df

No such comm: dfd93b873cb140d8a2741cce2f13e803
No such comm: f1d5d65b3d1e4a15b2ea912b88281f7d


                           Vendor Name   PO Number    PO Approver  \
0                             ExLibris    P1410545  Denise Rivers   
1                             ExLibris    P1410550  Denise Rivers   
2      Alliant Insurance Services, Inc  4500219360  Denise Rivers   
3      Alliant Insurance Services, Inc  4500219357  Denise Rivers   
4                                  EMC  4500147774  Denise Rivers   
...                                ...         ...            ...   
30412                  Statestore, Inc    GP120082     Luke Adams   
30413         Western Blue Corporation    GP120082   Timothy Rios   
30414           PATRIOT IT CORPORATION    HD250134  Shannon Joyce   
30417        Pacific Copier & Computer    IN120018  Rachel Martin   
30419     Technology Integration Group  4500198630     Evan Lyons   

      PO Order Date PO Approval Date  \
0        2012-02-08       2012-02-28   
1        2012-02-08       2012-02-28   
2        2015-06-19       2015-07-09   
3        20

In [7]:
df_cp=df.copy(deep=True)

In [8]:
# df.shape,df_cp.shape

((10148, 9), (10148, 9))

In [9]:
def add_fakedata(df,dist):
    np.random.seed(42)
    df_country=np.random.choice(list(dist.keys()), size=df.shape[0], p=list(dist.values()))
    return df_country

In [9]:
# 1-sum(dist.values())

In [12]:
dist = {
    'Czech Republic': 0.10,
    'Czechoslovkia': 0.01,
    'Czechia': 0.005,
    'Peru': 0.10,
    'Peruu':0.01,
    'China':0.40,
    'United States':0.20,
    'Singapore': 0.175
}
df_country=add_fakedata(df,dist)

In [17]:
# df.shape,df_cp.shape

((10148, 9), (10148, 10))

In [18]:
df.loc[:,'Country']=df_country

In [28]:
replacements = {'Czechoslovkia': 'Czech Republic', 'Czechia': 'Czech Republic', 'Peruu': 'Peru'}
df.loc[:,'Country_fixed']=df['Country'].replace(replacements)

In [57]:
def plot_histogram(col):
    print(df[col].value_counts(dropna=False))
    print(round(df[col].value_counts(normalize=True,dropna=False)*100),2)
    fig=px.histogram(df,x=col).update_xaxes(categoryorder='total descending')
    return fig

In [58]:
fig=plot_histogram('Country')
fig

China             4129
United States     1992
Singapore         1707
Czech Republic    1050
Peru              1030
Czechoslovkia      104
Peruu               89
Czechia             47
Name: Country, dtype: int64
China             41.0
United States     20.0
Singapore         17.0
Czech Republic    10.0
Peru              10.0
Czechoslovkia      1.0
Peruu              1.0
Czechia            0.0
Name: Country, dtype: float64 2


In [59]:
fig=plot_histogram('Country_fixed')
fig

China             4129
United States     1992
Singapore         1707
Czech Republic    1201
Peru              1119
Name: Country_fixed, dtype: int64
China             41.0
United States     20.0
Singapore         17.0
Czech Republic    12.0
Peru              11.0
Name: Country_fixed, dtype: float64 2


In [61]:
df.to_csv(r'C:\Users\User\Desktop\myGPT\data\17052023_ver1_cleansed_0.csv',index=False)